In [17]:
import re
import pandas as pd
from datetime import datetime
import numpy as np

In [18]:
# create paths to each df to be parsed and aggregated
comment_path = "../Data/cleaned_wsb_comments.csv"
post_path = "../Data/cleaned_wsb_posts.csv"
ticker_path = "../Data/final_ticker_list.csv"

In [19]:
# create dfs for each source
%time comment_df = pd.read_csv(comment_path)
%time post_df = pd.read_csv(post_path)
%time ticker_df = pd.read_csv(ticker_path)

CPU times: user 29.1 s, sys: 2.98 s, total: 32.1 s
Wall time: 32.5 s
CPU times: user 4.15 s, sys: 384 ms, total: 4.54 s
Wall time: 4.65 s
CPU times: user 2.04 ms, sys: 312 µs, total: 2.35 ms
Wall time: 7.18 ms


In [20]:
# create copies of the large dfs so we dont have to read them in multiple times while testing
post_title_count_df = post_df # to count and save tickers in the post titles
post_selftext_count_df = post_df # to count and save tickers in the post selftext (body)
comment_body_count_df = comment_df # to count and save tickers in the comment body

In [21]:
# convert the utc date to standard us format (left it labeled utc, oh well)
comment_df["created_utc"] = pd.to_datetime(comment_df["created_utc"], unit="s")
post_df["created_utc"] = pd.to_datetime(post_df["created_utc"], unit="s")
post_title_count_df["created_utc"] = pd.to_datetime(post_df["created_utc"], unit="s")
post_selftext_count_df["created_utc"] = pd.to_datetime(post_df["created_utc"], unit="s")
comment_body_count_df["created_utc"] = pd.to_datetime(comment_df["created_utc"], unit="s")

In [22]:
# loop over all the tickers in our ticker list, and parse each target string in each df and append new row as ticker, mention
# str to upper bc all our tickers are upper
%%time
for ticker in ticker_df["Tickers"]:
    post_title_count_df[ticker] = np.where(post_title_count_df['title'].str.upper().str.contains(rf'\s({ticker})\s', na=False), 1, 0)
    post_selftext_count_df[ticker] = np.where(post_selftext_count_df['selftext'].str.upper().str.contains(rf'\s({ticker})\s', na=False), 1, 0)
    comment_body_count_df[ticker] = np.where(comment_body_count_df['body'].str.upper().str.contains(rf'\s({ticker})\s', na=False), 1, 0)

CPU times: user 32min 48s, sys: 5min 27s, total: 38min 16s
Wall time: 40min 58s


In [23]:
# change the date again for agg purposes
post_title_count_df["date"] = post_title_count_df["created_utc"].dt.date 
post_selftext_count_df["date"] = post_selftext_count_df["created_utc"].dt.date
comment_body_count_df["date"] = comment_body_count_df["created_utc"].dt.date

In [32]:
# check all the dfs
# post_title_count_df.head()
# post_selftext_count_df.head()
# comment_body_count_df.head()

In [29]:
# group dfs by date, and sum all mentions
comment_body_count_df_agg = comment_body_count_df.groupby('date')[ticker_df["Tickers"]].sum()
post_title_count_df_agg = post_title_count_df.groupby('date')[ticker_df["Tickers"]].sum()
post_selftext_count_agg = post_selftext_count_df.groupby('date')[ticker_df["Tickers"]].sum()

In [30]:
# save un aggregated df, might be useful later, not sure yet
post_title_count_df.to_csv("../Data/post_title_count_df.csv")
post_selftext_count_df.to_csv("../Data/post_selftext_count_df.csv")
comment_body_count_df.to_csv("../Data/comment_body_count_df.csv")

In [31]:
# save all agg by day dfs
# comment_body_count_df_agg.to_csv("../Data/comment_body_count_agg.csv")
# post_title_agg_df.to_csv("../Data/post_title_count_agg.csv")
# post_selftext_count_df_agg.to_csv("../Data/post_selftext_count_agg.csv")